In [ ]:
### In this notebook we add new-primers to the ABE primer list and re-order the library

In [1]:
import pandas as pd 
from os import listdir
from os.path import join
import sys
sys.path.append('../src')
from utils import *
import matplotlib.pyplot as plt
import pickle
from Bio import SeqIO

In [5]:
out_path = '../../data/output/sg_Finder'
mrna_path = '../../data/MANE'
fpath = '../../data/output/sg_out/'

# Load all expressed transcripts
ess_genelist = pd.read_csv('../../data/output/k562_comm_ess.txt', header =None)
neutral_genelist = pd.read_csv('../../data/output/k562_neutral.txt', header =None)

fpath = '../../data/output/sg_out/'
list_gene = [i for i in listdir(fpath) if not i.startswith('.')]
list_gene = [i for i in list_gene if len(listdir(join(fpath,i))) > 5]
list_gene_ess = [i for i in list_gene if i in list(ess_genelist[0])]
list_gene_neutral = [i for i in list_gene if i in list(neutral_genelist[0])]

In [6]:
# How many genes not have any sgRNA detected from CHOPCHOP
# for gene_folder in join(out_path,gene,)
# 6 genes are first filtered out
inter_gene =list(set(ess_genelist[0]).intersection(set(list_gene_ess)))
left_out_genes = [ i for i in list(set(ess_genelist[0]))  if i not in list_gene_ess]

In [7]:
print(f'{len(left_out_genes)} genes didnt have any sgRNA\
generated by CHOPCHOP:{left_out_genes}')
### Usually because the isoforms varies too much that CHOPCHOP can't find a union. Tested SARS1 on website version.

6 genes didnt have any sgRNAgenerated by CHOPCHOP:['FARSB', 'LRR1', 'RPS9', 'GINS1', 'POLD3', 'SARS1']


In [8]:
### Filtered genes. They are the same across 11nt window and 13nt window
df_filter = pd.read_csv(join(out_path, 'ess_15','filter_gene.csv'), index_col =0)
df_filter = df_filter.set_index('gene')

***Get synonymous and stop codon sgRNA for ess and neutral genes***

In [9]:
# Read no primer list
ABE_sg = SeqIO.parse('../../data/output/primers/rREsg_ABE.fasta','fasta')
abe_sg = [str(i.seq) for i in ABE_sg]
# Dual_sg = SeqIO.parse('../data/output/primers/rREsg_Dual.fasta','fasta')
# dual_sg = [str(i.seq) for i in Dual_sg]
abe_mis_ety = pickle.load(open('ABE_MIS_ETY.pkl','rb'))

In [10]:
final_abe = abe_sg+abe_mis_ety

## ***Add primers***

In [11]:
rABE_noprimer_list,rABE_pass_list = get_noprimer_list(final_abe)
print(f'{len(rABE_noprimer_list)} ABE sg groups ')

1739 ABE sg groups 


In [12]:
write_noprime_list('ABE_order2.fasta',final_abe)

In [13]:
# F = 'GGGATCACTTACTACTTCCG'
# R = 'GGTACCCTATCACTTAACCG'
### We use the CBE primer to order this library
list_primers = []

F = 'CACTAGAGTAATCGCTACCG'
R = 'GGGCTACTAGACATAACTCG'

list_primers.append(F)
list_primers.append(R)

Fwd,Rev,oligoTM, primerTM = calc_best_primerTM(F,R, rABE_noprimer_list)
ABE_final_seq = attach_group_primer(rABE_pass_list,Fwd[:18],Rev[:18])
df_abe_final = pd.DataFrame({'sequence':ABE_final_seq})
df_abe_final['sequence'] = df_abe_final['sequence'].apply(lambda x:''.join(x))
df_abe_final = df_abe_final.reset_index()
df_abe_final['name'] = 'ABE_'+df_abe_final['index'].astype(str)

In [15]:
from itertools import combinations
for (i,j) in combinations(list_primers,2):
    print(p3.calcHeterodimer(i,j))

ThermoResult(structure_found=True, tm=-11.30, dg=-3947.59, dh=-39400.00, ds=-114.31)


In [16]:
df_abe_final[['name','sequence']].to_csv('../data/output/ABE_order.csv',index = False)